In [1]:
#IMDB scraper

In [125]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [133]:
#quick function to get text from between brackets
#turns out I could have just used .string ><
def findtext(w):
    w = str(w)
    w = w[w.find('>')+1:w.find('<', w.find('>'), len(w))]
    return w

#quick function to get money - does not convert to string at this time
def findmoney(w):
    w = str(w)
    w = w[w.find('$')+1:w.find('<', w.find('$'), len(w))]
    return w

In [134]:
###THIS IS THE FULL SCRAPE OF A PAGE###

def Get_Movie(url):
    # Request the page and use BeautifulSoup to extract the contents
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #get title
    title = soup.find(attrs={'data-testid':'hero-title-block__title'})
    title = findtext(title)

    #get release data and rating (PG, PG13, etc)
    DR = soup.find_all(attrs={'class':'TitleBlockMetaData__ListItemText-sc-12ein40-2 jedhex'})
    date = findtext(DR[0])
    rating = findtext(DR[1])

    #get a LIST of genres
    gen = soup.find(attrs={'data-testid':'storyline-genres'}) 
    gen = gen.find_all('a')
    genres = []
    for t in range(len(gen)):
        genres.append(findtext(gen[t]))

    #get box office info
    #budget often has '(estimated)' in it
    box = soup.find(attrs={'data-testid':'title-boxoffice-section'})
    budget = box.find(attrs={'data-testid':'title-boxoffice-budget'}) 
    budget = str(budget)
    budget = findmoney(budget)
    dom = box.find(attrs={'data-testid':'title-boxoffice-grossdomestic'})
    dom = findmoney(dom)
    ww = soup.find(attrs={'data-testid':'title-boxoffice-cumulativeworldwidegross'}) 
    ww = findmoney(ww)

    #get the description
    story = soup.find(attrs={'data-testid':'Storyline'}) 
    desc = story.find(attrs={'class':'ipc-html-content ipc-html-content--base'})
    desc = desc.div
    desc = findtext(desc)

    #get top tags
    #only shows top few tags, usually list tag is to full list which can be in hundreds
    kw = story.find(attrs={'data-testid':'storyline-plot-keywords'}) 
    kw = kw.find_all('span')
    keywords = []
    for t in range(len(kw)):
        keywords.append(findtext(kw[t]))
    if 'more' in keywords[-1]:
        keywords = keywords[:-1]

    #get runtime in minutes
    rt = soup.find(attrs={'data-testid':'title-techspec_runtime'}) 
    rt = rt.find('div')
    hrs = findtext(rt)
    rt = str(rt)
    mins = rt[::-1]
    mins = mins[7:]
    mins = mins[mins.find('<')+1:mins.find('>', mins.find('<'), len(mins))]
    runtime = int(hrs)*60 + int(mins)

    #put all that info in a list and return it 
    #NOTE: genres and keywords are both still list objects
    obs = [title,date,rating, genres, budget,dom,ww,desc,keywords, runtime]
    
    return obs

In [ ]:
#ok now pull in our list of movies
movies = pd.read_csv('FilmsList.csv')
movies = movies['Link']

In [136]:
#instantiate dataframe
data = pd.DataFrame(columns = ['Title', 'Release_Date', 'ViewRating', 'Genres', 'Budget', 'DomesticGross', 
                              'WorldwideGross', 'Description', 'Keywords', 'Runtime'])
#scrape provided pages
for this in movies:
    obs = Get_Movie('https://www.imdb.com{}'.format(this))
    data.loc[len(data)] = obs
        

In [137]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           2 non-null      object
 1   Release_Date    2 non-null      object
 2   ViewRating      2 non-null      object
 3   Genres          2 non-null      object
 4   Budget          2 non-null      object
 5   DomesticGross   2 non-null      object
 6   WorldwideGross  2 non-null      object
 7   Description     2 non-null      object
 8   Keywords        2 non-null      object
 9   Runtime         2 non-null      object
dtypes: object(10)
memory usage: 176.0+ bytes
None
